# 데이터 불러오기

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression # y=a * 총세대수 + b 의 회귀식을 만듬

import pandas as pd

train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sub = pd.read_csv("data/sample_submission.csv")
age = pd.read_csv("data/age_gender_info.csv")


train.shape, test.shape, sub.shape, age.shape


((2952, 15), (1022, 14), (150, 2), (16, 23))

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          1022 non-null   object 
 1   총세대수                          1022 non-null   int64  
 2   임대건물구분                        1022 non-null   object 
 3   지역                            1022 non-null   object 
 4   공급유형                          1022 non-null   object 
 5   전용면적                          1022 non-null   float64
 6   전용면적별세대수                      1022 non-null   int64  
 7   공가수                           1022 non-null   float64
 8   자격유형                          1020 non-null   object 
 9   임대보증금                         842 non-null    object 
 10  임대료                           842 non-null    object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  980 non-null    float64
 12  도보 10분거리 내 버스정류장 수            1022 non-null   float64
 13  단지내

# 결측치 처리
## 1) 결측치 컬럼 확인
* train : 임대보증금, 임대료 , 도보 10분 거리 내 지하철역 수, 도보 10분거리 내 버스정류장 수
* test : 자격유형, 임대보증금, 임대료 , 도보 10분 거리 내 지하철역 수

In [11]:
train.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

In [13]:
test.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              2
임대보증금                           180
임대료                             180
도보 10분거리 내 지하철역 수(환승노선 수 반영)     42
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
dtype: int64

## 2) 결측치 처리

## 오류 반영

### 2. 단지 코드 2개로 부여

In [51]:
test.isna().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            2
임대보증금         180
임대료           180
10분내지하철수       38
10분내버스정류장수      0
단지내주차면수         0
dtype: int64

In [52]:
train['지역'].value_counts()

경기도        602
경상남도       369
부산광역시      328
대전광역시      266
강원도        187
충청북도       163
충청남도       158
전라남도       147
광주광역시      141
경상북도       111
전라북도       109
대구광역시      102
제주특별자치도     91
서울특별시       64
울산광역시       34
세종특별자치시     24
Name: 지역, dtype: int64

In [53]:
test['지역'].value_counts()

경기도        251
강원도        106
부산광역시       88
경상남도        85
울산광역시       73
경상북도        58
대전광역시       52
충청북도        51
광주광역시       46
충청남도        41
전라북도        39
전라남도        37
대구광역시       36
제주특별자치도     26
세종특별자치시     19
Name: 지역, dtype: int64

## '지역' : test 데이터 셋에 '서울'이 없음.  (train에서 서울 뺌)


# 결측치 처리 :자격유형

In [54]:
test.loc[test['자격유형'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0


In [55]:
grouped = test.groupby(['단지코드','임대건물구분','지역','공급유형'])
group1 = grouped.get_group(('C2411','아파트','경상남도','국민임대'))
group1 # 자격유형이 A일것으로 판단

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
193,C2411,962,아파트,경상남도,국민임대,39.43,56,25.0,A,11992000,100720,0.0,2.0,840.0
194,C2411,962,아파트,경상남도,국민임대,39.72,336,25.0,A,11992000,100720,0.0,2.0,840.0
195,C2411,962,아파트,경상남도,국민임대,39.82,179,25.0,A,11992000,100720,0.0,2.0,840.0
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
197,C2411,962,아파트,경상남도,국민임대,51.93,150,25.0,A,21586000,171480,0.0,2.0,840.0


In [56]:
grouped = test.groupby(['단지코드','임대건물구분','지역','공급유형'])
group2 = grouped.get_group(('C2253','아파트','강원도','영구임대'))
group2 # 자격유형이 C일것으로 판단

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.0,C,3731000,83020,0.0,2.0,173.0
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.0,C,3731000,83020,0.0,2.0,173.0


In [57]:
test.loc[196,"자격유형"]='A'
test.loc[258,'자격유형']='C'

In [58]:
test.loc[[196,258],:]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,A,71950000,37470,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,C,2249000,44770,0.0,2.0,173.0


# 문자 숫자로 변경, 데이터 자료형 변환

In [59]:
print(sorted(train["자격유형"].unique()))
print(sorted(test["자격유형"].unique()))

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O']
['A', 'C', 'D', 'E', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']


In [60]:
mapping = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10
           , 'K':11, 'L':12, 'M':13, 'N':14, 'O':15}
print(train["자격유형"].isnull().sum())
print(test["자격유형"].isnull().sum())

0
0


In [61]:
train['자격유형']=train['자격유형'].map(mapping).astype(int)
test['자격유형']=test['자격유형'].map(mapping).astype(int)

In [62]:
train.head(3)
test.head(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,8,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,1,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,8,36048000,249930,0.0,2.0,683.0


## 우리가 분석하려는 데이터가 수치형이기 때문에 회귀분석

In [66]:
#train dataset의 train과 test로 나누기
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge
#10분내 정류장수 또는 지역 등등을 라벨 인코딩 해보기
sel = ['총세대수', '전용면적', '전용면적별세대수', '공가수',  '단지내주차면수', '자격유형']

X_train = train[sel]
y_train = train['등록차량수']

X_test = test[sel]


In [71]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

model = RandomForestRegressor(n_jobs=-1, random_state=42)
model.fit(X_train, y_train)  #학습
pred = model.predict(X_test) #예측


## 단지코드가 중복을 뺀 값들만 제출

In [ ]:
test['등록차량수']=pred

In [72]:
test['등록차량수']=pred
test['단지별차량수평균']=test.groupby('단지코드')['등록차량수'].transform(np.mean)

# test 1022개의 단지코드 중복을 빼주기 그리고 가장 첫번째 값만 남기기
test_new = test.drop_duplicates(['단지코드'],keep='first').reset_index()
# 결론적으로 150개만 남음.

#150개 코드별 차량수 평균을 sub_df로 만든다.
sub_df = test_new[['단지코드','단지별차량수평균']]

#컬럼명을 일치시킨다.
sub_df.columns =['code','num']
sub_df.to_csv('second_0715.csv',index=False)


In [79]:
sub_df.head()

,code,num
0,C1072,632.428750
1,C1128,1301.500000
2,C1456,650.947778
3,C1840,572.490000
4,C1332,1265.037500


# 오류조치

In [85]:
sub_df.loc[sub_df['code']== 'C2675']

,code,num


In [75]:
add_data = {'code':['C2675','C2335','C1327'],
           'num':[0,0,0]}
add_data = pd.DataFrame(add_data)
add_data

,code,num
0,C2675,0
1,C2335,0
2,C1327,0
